In [ ]:
%load_ext autoreload
%autoreload 2

This notebook highlights practical use cases for the FLamby integration into Fed-BioMed. A few datasets are here tested:
1. IXI Tiny
2. Heart-Disease
3. TCGA-BRCA
4. Synthetic
5. ISIC

For one FLamby dataset (LIDC-IDRI), we noticed conflicts over few packages including Torch, which cannot be easily resolved by changing package versions but need changes in the code (either on FLamby or Fed-BioMed side). This situation shows a certain complexity regarding the interoperability and has to be taken into account for future datasets that will be part of the suite.

### 1. Fed-IXI

In [ ]:
from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import AdamW

from flamby.datasets.fed_ixi import (Baseline as ixi_baseline,
                                     BaselineLoss as ixi_baseline_loss,
                                     Optimizer as ixi_optimizer,
                                     BATCH_SIZE as ixi_batch_size,
                                     LR as ixi_lr)

class UNetTrainingPlan(TorchTrainingPlan):
    # Init of UNetTrainingPlan
    def __init__(self, model_args: dict = {}):
        super(UNetTrainingPlan, self).__init__(model_args)
        
        self.model = ixi_baseline() # UNet model
        self.loss = ixi_baseline_loss() # Dice loss
        
        deps = ['from torch.optim import AdamW',
               'from flamby.datasets.fed_ixi import (Baseline as ixi_baseline,\
                BaselineLoss as ixi_baseline_loss,\
                Optimizer as ixi_optimizer)',]
        self.add_dependency(deps)
        
        self.optimizer = ixi_optimizer(self.parameters()) # AdamW
    
    def training_step(self, img, target):
        #this function must return the loss to backward it 
        output = self.model(img)
        loss = self.loss(output, target)
        return loss

In [ ]:
training_args = {
    'batch_size': ixi_batch_size,
    'lr': ixi_lr,
    'epochs': 1,
}

*train_transform_flamby* key in **training_args** can optionally be used to perform extra transformations on a flamby dataset.

As a reminder, flamby datasets are already internally handling a transformation through their dataloader (this internal transform
is the one officially used for the flamby benchmark). Thus, one should check what is already performed on the flamby side before
adding transforms through the researcher.

*train_transform_flamby* has to be defined as a list containing two elements:
- the first is the imports needed to perform the transformation
- the second is the Compose object that will be used to input the transform parameter of the flamby dataset federated class

Example:
```python
training_args = {
    ...,
    'train_transform_flamby':["from monai.transforms import (Compose, NormalizeIntensity, Resize,)",
                         "Compose([Resize((48,60,48)), NormalizeIntensity()])"]
}
```

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['ixi']
num_rounds = 1

exp = Experiment(tags=tags,
                 model_class=UNetTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

An experiment is a class that orchestrates the training processes that run on different nodes. The experiment has been here initialized with necessary arguments to inform nodes about how to process the training data based on a given model.

Let's run the experiment. According to the provided arguments, 1 training round should be completed on the nodes that you created (3 nodes here because there are 3 centers in the case of IXI).
Aggregated parameters are saved at the end of the experiment, and their state at every round can be loaded.

In [ ]:
exp.run()

### 2. Fed-Heart-Disease

In [ ]:
from flamby.datasets.fed_heart_disease import (Baseline as hd_baseline,
                                               BaselineLoss as hd_baseline_loss,
                                               Optimizer as hd_optimizer,
                                               BATCH_SIZE as hd_batch_size,
                                               LR as hd_lr)

from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import Adam

class FedHeartTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(FedHeartTrainingPlan, self).__init__(model_args)
        
        self.model = hd_baseline()
        self.loss = hd_baseline_loss()
        
        deps = ['from torch.optim import Adam',
                'from flamby.datasets.fed_heart_disease import (Baseline as hd_baseline,\
                BaselineLoss as hd_baseline_loss,\
                Optimizer as hd_optimizer)']
        self.add_dependency(deps)
        
        self.optimizer = hd_optimizer(self.parameters())
    
    def training_step(self, data, target):
        output = self.model(data)
        loss = self.loss(output, target)
        return loss

In [ ]:
training_args = {
    'batch_size': hd_batch_size,
    'lr': hd_lr,
    'epochs': 5,
}

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['hd']
num_rounds = 1

exp = Experiment(tags=tags,
                 model_class=FedHeartTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

In [ ]:
exp.run()

### 3. Fed-TCGA-BRCA

In [ ]:
from flamby.datasets.fed_tcga_brca import (Baseline as tc_baseline,
                                           BaselineLoss as tc_baseline_loss,
                                           Optimizer as tc_optimizer,
                                           BATCH_SIZE as tc_batch_size,
                                           LR as tc_lr)
from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import Adam

class FedTcgaBrcaTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(FedTcgaBrcaTrainingPlan, self).__init__(model_args)
        
        self.model = tc_baseline()
        self.loss = tc_baseline_loss()
        
        deps = ['from torch.optim import Adam',
               'from flamby.datasets.fed_tcga_brca import (Baseline as tc_baseline,\
                BaselineLoss as tc_baseline_loss,\
                Optimizer as tc_optimizer)']
        self.add_dependency(deps)
        
        self.optimizer = tc_optimizer(self.parameters())
    
    def training_step(self, data, target):
        output = self.model(data)
        loss = self.loss(output, target)
        return loss

In [ ]:
training_args = {
    'batch_size': tc_batch_size,
    'lr': tc_lr,
    'epochs': 5,
}

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['tc']
num_rounds = 1

exp = Experiment(tags=tags,
                 model_class=FedTcgaBrcaTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

In [ ]:
exp.run()

### 4. Fed-Synthetic

In [ ]:
from flamby.datasets.fed_synthetic import (Baseline as sy_baseline,
                                           BaselineLoss as sy_baseline_loss,
                                           Optimizer as sy_optimizer,
                                           BATCH_SIZE as sy_batch_size,
                                           LR as sy_lr)
from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import Adam

class FedSyntheticTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(FedSyntheticTrainingPlan, self).__init__(model_args)
        
        self.model = sy_baseline(model_args.get('input_dim', 10), model_args.get('output_dim', 1)) # specific to synthetic use case
        self.loss = sy_baseline_loss()
        
        deps = ['from torch.optim import Adam',
               'from flamby.datasets.fed_synthetic import (Baseline as sy_baseline,\
                BaselineLoss as sy_baseline_loss,\
                Optimizer as sy_optimizer)',]
        self.add_dependency(deps)
        
        self.optimizer = sy_optimizer(self.parameters())
    
    def training_step(self, data, target):
        #this function must return the loss to backward it 
        output = self.model(data)
        loss = self.loss(output, target)
        return loss

In [ ]:
training_args = {
    'batch_size': sy_batch_size,
    'lr': sy_lr,
    'epochs': 5,
}

model_args = {
    'input_dim': 10,
    'output_dim': 1,
}

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['sy']
num_rounds = 1

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=FedSyntheticTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

In [ ]:
exp.run()

### 5. Fed-ISIC

In [ ]:
from flamby.datasets.fed_isic2019 import (Baseline as is_baseline,
                                          BaselineLoss as is_baseline_loss,
                                          Optimizer as is_optimizer,
                                          BATCH_SIZE as is_batch_size,
                                          LR as is_lr
                                         )
from fedbiomed.common.training_plans import TorchTrainingPlan
from torch.optim import Adam

class FedISICTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(FedISICTrainingPlan, self).__init__(model_args)
        
        self.model = is_baseline()
        self.loss = is_baseline_loss()
        
        deps = ['from torch.optim import Adam',
               'from flamby.datasets.fed_isic2019 import (Baseline as is_baseline,\
                BaselineLoss as is_baseline_loss,\
                Optimizer as is_optimizer)',]
        self.add_dependency(deps)
        
        self.optimizer = Adam(self.parameters())
    
    def training_step(self, data, target):
        output = self.model(data)
        loss = self.loss(output, target)
        return loss

In [ ]:
training_args = {
    'batch_size': is_batch_size,
    'lr': is_lr,
    'epochs': 1,
}

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['is']
num_rounds = 1

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=FedISICTrainingPlan,
                 training_args=training_args,
                 round_limit=num_rounds,
                 aggregator=FedAverage(),
                )

In [ ]:
exp.run()